In [1]:
!python --version

Python 3.10.12


In [2]:
cd uni2ts

/home/user/energygpt/moirai/uni2ts


/home/user/miniconda3/envs/moirai/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
import torch
import matplotlib.pyplot as plt
import glob
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from tqdm.autonotebook import tqdm
import matplotlib.dates as mdates
from itertools import islice
from collections import defaultdict
import gluonts
from datetime import datetime

/tmp/ipykernel_3722396/4260485682.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
from uni2ts.eval_util.plot import plot_single
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.evaluation import evaluate_model

In [5]:
# Data pipelining
def get_batched_data_fn(sub_df,
    batch_size: int = 128, 
    context_len: int = 168, 
    horizon_len: int = 24):
    
    examples = defaultdict(list)
    num_examples = 0
    for start in range(0, len(sub_df) - (context_len + horizon_len), horizon_len):
      num_examples += 1
      #examples["country"].append(country)
      examples["inputs"].append(sub_df["y"][start:(context_end := start + context_len)].tolist())
      #examples["gen_forecast"].append(sub_df["gen_forecast"][start:context_end + horizon_len].tolist())
      #examples["week_day"].append(sub_df["week_day"][start:context_end + horizon_len].tolist())
      examples["outputs"].append(sub_df["y"][context_end:(context_end + horizon_len)].tolist())
      examples['inputs_ts'].append(sub_df.index[start:(context_end := start + context_len)])
      examples["outputs_ts"].append(sub_df.index[context_end:(context_end + horizon_len)])

    return examples

In [ ]:
def forecast_building(df):

    # Set numerical columns as float32
    for col in df.columns:
        # Check if column is not of string type
        if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
            df[col] = df[col].astype('float32')
    
    # Create the Pandas
    dataset = PandasDataset.from_long_dataframe(df, target="target", item_id="item_id")

    backtest_dataset = dataset
    prediction_length = 24  # Define your prediction length. We use 24 here since the data is of hourly frequency
    num_samples = 100

    model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small"),
    prediction_length=prediction_length,
    context_length=168,
    patch_size='auto',
    target_dim=1,
    feat_dynamic_real_dim=backtest_dataset.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=backtest_dataset.num_past_feat_dynamic_real,
)
    predictor = model.create_predictor(batch_size=32, device="cuda:0")

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=backtest_dataset,
        predictor=predictor,
        num_samples=num_samples
    )

    forecasts = list(forecast_it)
    tss = list(ts_it)

    evaluator = Evaluator()
    agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))

    res_all = []
    for ts, fc in zip(tss, forecasts):
        res = ts[ts.index.isin(fc.index)]
        res.columns = ['y_true']
        res.insert(1, 'y_pred', fc.median)        
        res_all.append(res)
        #print(gt.shape)
        #break
    res_all_df = pd.concat(res_all).sort_index()
    return res_all_df, agg_metrics, ts_metrics 


In [7]:
def process_building(df): 
    building_name = df.columns[0]
    df.columns = ['y']
    input_data = get_batched_data_fn(df, batch_size=500)
    # print(input_data)
    
    windows_all = []
    counter = 1
    for inputs_ts, inputs, outputs_ts, outputs in zip(input_data['inputs_ts'], 
                                                      input_data['inputs'], 
                                                      input_data['outputs_ts'], 
                                                      input_data['outputs']):
        
        input_df = pd.DataFrame({'timestamp': inputs_ts, 
                                 'target': inputs})
        
        output_df = pd.DataFrame({'timestamp': outputs_ts, 
                                 'target': outputs})
        combined = pd.concat([input_df, output_df], axis=0)
        combined['item_id'] = str(building_name) + '_' + str(counter)
        combined['item_id_no'] = counter
        counter += 1
        windows_all.append(combined)
        
    windows_all_df = pd.concat(windows_all)
    windows_all_df.timestamp = pd.to_datetime(windows_all_df.timestamp)
    windows_all_df.set_index('timestamp', inplace=True)
    windows_all_df.to_csv('test.csv')

    res, agg_metrics, ts_metrics = forecast_building(windows_all_df)
    return res, agg_metrics, ts_metrics

In [ ]:
def process_file(filename):
    df = pd.read_csv(filename)
    df = df.set_index(['timestamp'])

    if df.shape[1] < 2:
        return None
        
    print(datetime.now(), df.shape, flush=True)

    res_all = []
    agg_metrics_all = []
    ts_metrics_all = []
    
    i = 0
    for building_name in df.columns:
        print(datetime.now(), i, '/', len(df.columns), building_name, flush=True)
        df1 = df[[building_name]]

        res, agg_metrics, ts_metrics = process_building(df1)
        res['building'] = building_name
        res['filename'] = filename
        res_all.append(res)
        
        ts_metrics.insert(0, 'building', building_name)
        ts_metrics.insert(0, 'filename', filename)
        ts_metrics = ts_metrics.sort_values(['forecast_start'])
        ts_metrics_all.append(ts_metrics)
        
        agg_metrics_df = pd.DataFrame([agg_metrics])
        agg_metrics_df.insert(0, 'building', building_name)
        agg_metrics_df.insert(0, 'filename', filename)
        agg_metrics_all.append(agg_metrics_df)

        i += 1
        if i % 10 == 0:
            print(datetime.now(), 'Saving...')
            res_all_df = pd.concat(res_all).round(6)
            res_all_df = res_all_df.reset_index()
            res_all_df = res_all_df.rename(columns={res_all_df.columns[0]: "timestamp" })
            res_all_df.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

            ts_metrics_all_df = pd.concat(ts_metrics_all).round(6)
            ts_metrics_all_df.to_csv(f'../results/{dataset}/ts_metrics_{os.path.basename(filename)}', index=False)            

            agg_metrics_all_df = pd.concat(agg_metrics_all).round(6)            
            agg_metrics_all_df.to_csv(f'../results/{dataset}/agg_metrics_{os.path.basename(filename)}', index=False)            
    
    
    res_all_df = pd.concat(res_all).round(6)
    res_all_df = res_all_df.reset_index()
    res_all_df = res_all_df.rename(columns={res_all_df.columns[0]: "timestamp" })
    res_all_df.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)            

    ts_metrics_all_df = pd.concat(ts_metrics_all).round(6)    
    ts_metrics_all_df.to_csv(f'../results/{dataset}/ts_metrics_{os.path.basename(filename)}', index=False)            

    agg_metrics_all_df = pd.concat(agg_metrics_all).round(6)   
    agg_metrics_all_df.to_csv(f'../results/{dataset}/agg_metrics_{os.path.basename(filename)}', index=False)                

    return res_all_df, ts_metrics_all_df, agg_metrics_all_df

In [10]:
files_list = glob.glob('/home/user/New_Buildings_Datasets/Enernoc/csv-only/processed/*.csv')

dataset = 'Enernoc'
os.makedirs(f'../forecasts/{dataset}/', exist_ok = True)
os.makedirs(f'../results/{dataset}/', exist_ok = True)

for filename in files_list:
    print(datetime.now(), filename)
    results = process_file(filename)
    # if results is not None:
    #     results.to_csv(f'../forecasts/{dataset}/{os.path.basename(filename)}', index=False)
    print('')

2024-10-23 10:57:32.658389 /home/user/New_Buildings_Datasets/Enernoc/csv-only/processed/enernoc.csv
2024-10-23 10:57:32.781264 (8785, 100)
2024-10-23 10:57:32.782156 0 / 100 767


Running evaluation: 359it [00:00, 5737.61it/s]
/home/user/miniconda3/envs/moirai/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:105: RuntimeWarning: divide by zero encountered in scalar divide
  return np.mean(np.abs(target - forecast)) / seasonal_error
/home/user/miniconda3/envs/moirai/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWarning: divide by zero encountered in scalar divide
  metrics["ND"] = cast(float, metrics["abs_error"]) / cast(
/home/user/miniconda3/envs/moirai/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:154: RuntimeWarning: divide by zero encountered in scalar divide
  return numerator / seasonal_error
/home/user/miniconda3/envs/moirai/lib/python3.10/site-packages/gluonts/evaluation/metrics.py:105: RuntimeWarning: divide by zero encountered in scalar divide
  return np.mean(np.abs(target - forecast)) / seasonal_error
/home/user/miniconda3/envs/moirai/lib/python3.10/site-packages/gluonts/evaluation/_base.py:425: RuntimeWa

2024-10-23 10:57:39.476665 1 / 100 304


Running evaluation: 359it [00:00, 7637.46it/s]


2024-10-23 10:57:44.633441 2 / 100 399


...

2024-10-23 11:06:27.802895 98 / 100 765


Running evaluation: 359it [00:00, 7557.19it/s]


2024-10-23 11:06:32.951143 99 / 100 716


Running evaluation: 359it [00:00, 7505.51it/s]


2024-10-23 11:06:38.010426 Saving...

